In [1]:
import os
import time
import requests
import pandas as pd
from bs4 import BeautifulSoup as bs

from myfunc import text_clean #myfunc의 text_clean함수 가져오기

In [ ]:
?categoryNumber=001&pageNumber=2&pageSize=24 #24개짜리 한 페이지만 가져온 거임

In [5]:
#요청 & 파싱 파트 - 웹페이지를 파이썬으로 가져오는 과정
#24개짜리 한 페이지만 가져온 거임

url="https://www.yes24.com/product/category/bestseller"
payload=dict(categoryNumber='001',pageNumber=3,pageSize=24)
r=requests.get(url, params=payload)
soup = bs(r.content, 'lxml')
#soup #html yes24 페이지

soup = bs(r.content, 'lxml') -> HTML을 DOM 트리로 바꾸는 선언문 -> 부모–자식 관계 탐색 가능

- r.content : YES24 서버가 준 HTML 원본, 문자열 덩어리
- bs (BeautifulSoup) : HTML을 분석해주는 라이브러리, import해서 사용함
- lxml : HTML을 해석하는 엔진(parser) 이름, BeautifulSoup은 직접 해석 안 함 → 외주 줌, 부모–자식 구조로 정리

In [3]:
#soup.find_all("a") # a태그 결과 매우 많음

In [61]:
soup.select_one(".gd_name")
#정리된 DOM 트리에서 class가 gd_name인 처음 1개 요소 찾아줘

<a class="gd_name" href="/product/goods/172506508" onclick="wiseLogV2('BS', '001_005_001', ''); ">돈의 방정식</a>

In [62]:
soup.select_one(".gd_name").get_text()

'돈의 방정식'

In [10]:
soup.select_one("span.authPub.info_auth").get_text() #-> css
#span태그 안에 클래스가 2개(authPub.info_auth) 있는 것을 찾아서 가져와라

'\n이광수 저\r\n                            '

- css에서 셀렉터가 특히 중요함!!

### html, css 공부
문서 - 04폴더 들어가서 - cmd창 열고 - code . 쳐서 vs code 들어감 - 쓰고 alt+l+o 누르면 실행

### yes24 페이지 들어가서 -> f12 키 누르면 html창 나옴 -> 왼쪽 위에 있는 화살표 있는 버튼 클릭하면 마우스 갖다 댄 부분(제목, 저자 등..) html창 나옴

### 수집할 것들 -> 종류, 책 제목, 상세 페이지 링크, 저자/역/글그림/그림, 출판사, 출간일, 가격, 회원리뷰수, 평점

- soup : HTML 원문을 부모–자식 관계가 있는 DOM 구조로 바꿔 놓은 것
- select_one : DOM 트리에서 조건에 맞는 요소 “하나”를 찾아줘
- .get_text() : 태그 안의 글자만

```python
kind = text_clean(soup.select_one(".gd_res").get_text())
```
soup.select_one(".gd_res") → class가 gd_res인 요소 하나 찾아라

In [5]:
soup.select_one(".gd_res").get_text()

'[도서]'

In [6]:
#책 종류
kind=text_clean(soup.select_one(".gd_res").get_text())

#책 제목
book_title=soup.select_one(".gd_name").get_text()

# 책 상세 페이지 링크 구하기
#soup.select_one(".gd_name")
# 책 상세 페이지 주소 - 직접 가져옴
#https://www.yes24.com/product/goods/172506508

# 책 상세 페이지 링크
detail_link="https://www.yes24.com" + soup.select_one(".gd_name")['href']

#저자, 역자 함께 나옴 - 같은 태그에 클래스 2개
author=soup.select_one(".authPub.info_auth").get_text().replace("\n",'').replace("\r",'')

# 출판사
publisher=soup.select_one(".authPub.info_pub").get_text()

#출간일
pub_date=soup.select_one(".authPub.info_date").get_text()

#책 가격
# class가 info_row & info_price인 div 안에 있는 -> em 태그 중 class가 yes_b
book_price=int(soup.select_one("div.info_row.info_price em.yes_b").get_text().replace(",",''))

#리뷰 수
n_reviews=int(soup.select_one(".txC_blue").get_text())

#리뷰 평점
# class가 rating_grade인 태그 안에 있는 em 태그 중에서 class가 yes_b인 것
rating=float(soup.select_one(".rating_grade em.yes_b").get_text())

print(kind, book_title, detail_link, author, publisher, pub_date, book_price, n_reviews, rating)

도서 하루 한 장 나의 어휘력을 위한 필사 노트 https://www.yes24.com/product/goods/125557465 유선경 저                             위즈덤하우스 2024년 10월 21420 883 9.5


### 예시) 리뷰 평점 -> .rating_grade em.yes_b
- class가 rating_grade인 태그 안에 있는 em 태그 중에서 class가 yes_b인 것
- class="rating_grade" 안에 있는 → ```<em class="yes_b">```

- html 모양
```python
<div class="rating_grade">
    <em class="yes_b">9.6</em>
</div>
```

- 돔트리 모양
div.rating_grade        ← 부모
 └─ em.yes_b            ← 자식
     └─ "9.6"           ← 텍스트

In [52]:
# yesBestList의 자식 li만 필요함
soup.select("#yesBestList > li")[0] #yesBestList의 자식 li들 중 첫 번째 = 돈의 방정식

<li class="" data-goods-no="172506508" data-iy-no="0" data-statgb="02">
<div class="itemUnit">
<div class="item_img">
<div class="img_canvas">
<div class="img_upper">
<em class="ico rank">1</em>
<span class="rank_info rank_up">
<span class="ico"></span><em class="txt rank">3</em><em class="txt">위 상승</em>
</span>
</div>
<span class="img_item">
<span class="img_grp">
<a class="lnk_img" href="/product/goods/172506508" onclick="wiseLogV2('BS', '001_005_001', ''); ">
<em class="img_bdr">
<img alt="돈의 방정식" border="0" class="lazy" data-original="https://image.yes24.com/goods/172506508/L" src="https://image.yes24.com/momo/Noimg_L.jpg"/>
</em>
</a>
</span>
</span>
</div>
<div class="img_btn">
<a class="btnC btn_preview" href="javascript:yes24GU.openPreviewCheck(172506508); wiseLogV2('BS', '001_005_011', '');"><span class="bWrap"><em class="txt">미리보기</em></span></a>
</div>
</div>
<div class="item_info">
<div class="info_row info_keynote">
<span class="gd_keynote" id="spanGdKeynote">
<a class="ic

In [7]:
# 반복문으로 전체 출력

for item in soup.select("#yesBestList>li"):
    #책 종류
    kind=text_clean(item.select_one(".gd_res").get_text())

    #책 제목
    book_title=item.select_one(".gd_name").get_text()

    # 책 상세 페이지 링크
    detail_link="https://www.yes24.com" + item.select_one(".gd_name")['href']

    #저자, 역자 함께 나옴
    author=item.select_one(".authPub.info_auth").get_text().replace("\n",'').replace("\r",'')

    # 출판사
    publisher=item.select_one(".authPub.info_pub").get_text()

    #출간일
    pub_date=item.select_one(".authPub.info_date").get_text()

    #책 가격
    book_price=int(item.select_one("div.info_row.info_price em.yes_b").get_text().replace(",",''))

    #리뷰 수
    n_reviews=int(item.select_one(".txC_blue").get_text().replace(",",'')) if item.select_one(".txC_blue")!=None else 0

    #리뷰 평점
    rating=float(item.select_one(".rating_grade em.yes_b").get_text()) if item.select_one(".rating_grade em.yes_b")!=None else 0
    
    print(kind, book_title, detail_link, author, publisher, pub_date, book_price, n_reviews, rating)

도서 하루 한 장 나의 어휘력을 위한 필사 노트 https://www.yes24.com/product/goods/125557465 유선경 저                             위즈덤하우스 2024년 10월 21420 883 9.5
도서 흔한남매 21 https://www.yes24.com/product/goods/167627172 흔한남매 원저/백난도 글/유난희 그림/흔한컴퍼니 감수                             미래엔아이세움 2025년 12월 15120 104 9.7
도서 뇌는 어떻게 나를 조종하는가 https://www.yes24.com/product/goods/171432215 크리스 나이바우어 저/김윤종 역                             클랩북스 2026년 01월 16920 29 9.6
도서 쓰면서 바로 써먹는 초등 어휘 따라 쓰기 https://www.yes24.com/product/goods/169629856 올바른초등교육연구소 글/고도연 그림                             피넛 2026년 01월 15120 25 10.0
도서 어른의 품격을 채우는 100일 필사 노트 https://www.yes24.com/product/goods/142687238 김종원 저                             청림Life 2025년 03월 18000 249 9.8
도서 내가 선택할 수 있는 품격 있는 태도에 관하여 https://www.yes24.com/product/goods/169689075 김종원 저                             오아시스 2025년 12월 17100 74 9.9
도서 2026 심우철 실전 동형 모의고사 Season 1 https://www.yes24.com/product/goods/171015710 심우철 저                             에스티유니타스 2025년 12월 13500 49 10.0
도서 떠난 것은 돌아

In [57]:
soup.select_one(".gd_name")['href']

'/product/goods/172506508'

In [58]:
soup.select_one(".gd_name")['href'].split("/")[-1]

'172506508'

In [8]:
# 전체 모아서 데이터프레임(DataFrame)으로 출력하기

result={}

for item in soup.select("#yesBestList>li"):
    book_id=item.select_one(".gd_name")["href"].split("/")[-1]
    #책 종류
    kind=text_clean(item.select_one(".gd_res").get_text())

    #책 제목
    book_title=item.select_one(".gd_name").get_text()

    # 책 상세 페이지 링크
    detail_link="https://www.yes24.com" + item.select_one(".gd_name")['href']

    #저자, 역자 함께 나옴
    author=item.select_one(".authPub.info_auth").get_text().replace("\n",'').replace("\r",'')

    # 출판사
    publisher=item.select_one(".authPub.info_pub").get_text()

    #출간일
    pub_date=item.select_one(".authPub.info_date").get_text()

    #책 가격
    book_price=int(item.select_one("div.info_row.info_price em.yes_b").get_text().replace(",",''))

    #리뷰 수
    n_reviews=int(item.select_one(".txC_blue").get_text().replace(",",'')) if item.select_one(".txC_blue")!=None else 0

    #리뷰 평점
    rating=float(item.select_one(".rating_grade em.yes_b").get_text()) if item.select_one(".rating_grade em.yes_b")!=None else 0
    
    keys=("book_id", "kind", "book_title",'detail_link', 'author', 'publisher', 'pub_date', 'book_price','n_reviews', 'rating')
    values=(book_id,kind, book_title, detail_link, author, publisher, pub_date, book_price, n_reviews, rating)
    
    for key, value in zip(keys,values):
        result.setdefault(key, []).append(value)
        
df=pd.DataFrame(result)
df

    

,book_id,kind,book_title,detail_link,author,publisher,pub_date,book_price,n_reviews,rating
0,125557465,도서,하루 한 장 나의 어휘력을 위한 필사 노트,https://www.yes24.com/product/goods/125557465,유선경 저,위즈덤하우스,2024년 10월,21420,883,9.5
1,167627172,도서,흔한남매 21,https://www.yes24.com/product/goods/167627172,흔한남매 원저/백난도 글/유난희 그림/흔한컴퍼니 감수 ...,미래엔아이세움,2025년 12월,15120,104,9.7
2,171432215,도서,뇌는 어떻게 나를 조종하는가,https://www.yes24.com/product/goods/171432215,크리스 나이바우어 저/김윤종 역,클랩북스,2026년 01월,16920,29,9.6
3,169629856,도서,쓰면서 바로 써먹는 초등 어휘 따라 쓰기,https://www.yes24.com/product/goods/169629856,올바른초등교육연구소 글/고도연 그림,피넛,2026년 01월,15120,25,10.0
4,142687238,도서,어른의 품격을 채우는 100일 필사 노트,https://www.yes24.com/product/goods/142687238,김종원 저,청림Life,2025년 03월,18000,249,9.8
5,169689075,도서,내가 선택할 수 있는 품격 있는 태도에 관하여,https://www.yes24.com/product/goods/169689075,김종원 저,오아시스,2025년 12월,17100,74,9.9
6,171015710,도서,2026 심우철 실전 동형 모의고사 Season 1,https://www.yes24.com/product/goods/171015710,심우철 저,에스티유니타스,2025년 12월,13500,49,10.0
7,174667883,도서,떠난 것은 돌아오지 않는다,https://www.yes24.com/product/goods/174667883,줄리언 반스 저/정영목 역,다산북스,2026년 01월,16200,19,9.7
8,171520765,도서,2026 선재국어 결승선 봉투 모의고사,https://www.yes24.com/product/goods/171520765,이선재 편저,수비니겨,2026년 01월,25200,29,10.0
9,153064968,도서,트렌드 코리아 2026,https://www.yes24.com/product/goods/153064968,"김난도, 전미영, 최지혜, 권정윤, 한다혜 저 외 7명 ...",미래의창,2025년 09월,18000,242,9.2


# 저자 분리하기

In [14]:
soup.select("#yesBestList>li")[7].select_one(".authPub.info_auth").get_text()

'\n줄리언 반스 저/정영목 역\r\n                            '

In [16]:
soup.select("#yesBestList>li")[1].select_one(".authPub.info_auth").get_text().split("/")

['\n흔한남매 원저', '백난도 글', '유난희 그림', '흔한컴퍼니 감수\r\n                            ']

In [28]:
for name_by_role in soup.select("#yesBestList>li")[1].select_one(".authPub.info_auth").get_text().split("/"):
    print(name_by_role.split())

['흔한남매', '원저']
['백난도', '글']
['유난희', '그림']
['흔한컴퍼니', '감수']


In [32]:
result={}
for name_by_role in soup.select("#yesBestList>li")[1].select_one(".authPub.info_auth").get_text().split("/"):
    name, role=name_by_role.split()
    print(name, role)
    result[role]=name #딕셔너리에 키, 밸류 값 넣어주기
    
result

흔한남매 원저
백난도 글
유난희 그림
흔한컴퍼니 감수


{'원저': '흔한남매', '글': '백난도', '그림': '유난희', '감수': '흔한컴퍼니'}

In [37]:
result={}
for name_by_role in soup.select("#yesBestList>li")[2].select_one(".authPub.info_auth").get_text().split("/"):
    temp=name_by_role.split()
    name=" ".join(temp[:-1])
    role=temp[-1]
    result[role]=name
result

{'저': '크리스 나이바우어', '역': '김윤종'}

In [18]:
# 저자 여러 명인 경우
authors=soup.select("#yesBestList>li")[9].select_one(".authPub.info_auth").get_text()
authors

'\n김난도, 전미영, 최지혜, 권정윤, 한다혜 저 외 7명\r\n                                        \n정보 더 보기/감추기\n\n\n\n김난도\n전미영\n최지혜\n권정윤\n한다혜\n이혜원\n이수진\n서유현\n전다현\n이준영\n이향은\n김나은\n\n\n\n\n'

In [24]:
if "정보 더 보기/감추기" in authors:
    temp=authors.split("정보 더 보기/감추기")
    print(temp[1])





김난도
전미영
최지혜
권정윤
한다혜
이혜원
이수진
서유현
전다현
이준영
이향은
김나은







In [23]:
if "정보 더 보기/감추기" in authors:
    temp=authors.split("정보 더 보기/감추기")
    print(temp[1].strip().split("\n"))

['김난도', '전미영', '최지혜', '권정윤', '한다혜', '이혜원', '이수진', '서유현', '전다현', '이준영', '이향은', '김나은']


In [26]:
authors=soup.select("#yesBestList>li")[13].select_one(".authPub.info_auth").get_text()
if "정보 더 보기/감추기" in authors:
    temp=authors.split("정보 더 보기/감추기")
    authors=temp[1].strip().split("\n")
    print(authors)

['이반 투르게네프', '토마스 하디', '기 드 모파상', '셔우드 앤더슨', '안톤 파블로비치 체호프', '허먼 멜빌', '에드가 앨런 포', '레프 톨스토이', '마이너스']


## 저자 분리하기 최종 코드 (합침)

In [42]:
author_result=[]

for item in soup.select("#yesBestList>li"): #책 리스트에서 한 권씩 가져옴
    result={} # 1권의 저자 정보가 모임
    authors=item.select_one(".authPub.info_auth").get_text() # 저자 뽑음
    if "정보 더 보기/감추기" in authors:
        temp=authors.split("정보 더 보기/감추기")
        authors=temp[1].strip().split("\n")
        result['저']=author
    else:
        for name_by_role in item.select_one(".authPub.info_auth").get_text().split("/"):
            temp=name_by_role.split()
            name=" ".join(temp[:-1])
            role=temp[-1]
            result[role]=name
    
    author_result.append(result)
    
df=pd.DataFrame(author_result)
df

,저,원저,글,그림,감수,역,편저
0,유선경,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,흔한남매,백난도,유난희,흔한컴퍼니,NaN,NaN
2,크리스 나이바우어,NaN,NaN,NaN,NaN,김윤종,NaN
3,NaN,NaN,올바른초등교육연구소,고도연,NaN,NaN,NaN
4,김종원,NaN,NaN,NaN,NaN,NaN,NaN
5,김종원,NaN,NaN,NaN,NaN,NaN,NaN
6,심우철,NaN,NaN,NaN,NaN,NaN,NaN
7,줄리언 반스,NaN,NaN,NaN,NaN,정영목,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,이선재
9,ETS 저,NaN,NaN,NaN,NaN,NaN,NaN


# yes24 종합 베스트 999권 자료 수집해서 csv파일로 저장하기
- book_data/
- yes24_수집일_best.csv
- 수집일은 datatime에서 오늘 날짜 추출해서 파일명에 포함시키기

In [3]:
import os
import time
import requests
import pandas as pd
from bs4 import BeautifulSoup as bs

from myfunc import text_clean #myfunc의 text_clean함수 가져오기

In [4]:
from datetime import datetime

In [5]:
today=datetime.today()
print(today)

2026-01-27 15:33:40.905948


In [6]:
today= f"{today.year}{today.month:02d}{today.day:02d}"
today

'20260127'

In [23]:
all_result = []
for page in range(1, 10):
    print(f"{page}/{9}   수집중", end="\r")
    url = "https://www.yes24.com/product/category/bestseller"
    payload = dict(categoryNumber="001", pageNumber=page, pageSize=120)
    r = requests.get(url, params=payload)
#     print(r.status_code)
    soup = bs(r.content, "lxml")
    
    
    for item in soup.select("#yesBestList > li"):
        result = {}
        book_id = item.select_one(".gd_name")["href"].split("/")[-1]
        # 책종류
        kind = text_clean(item.select_one(".gd_res").get_text())
        # 책제목
        book_title = item.select_one(".gd_name").get_text()
        # 상세페이지링크 
        detail_link = "https://www.yes24.com" + item.select_one(".gd_name")["href"]
        # 저자/역자
        author = auth_extract(item.select_one(".authPub.info_auth").get_text())
        # 출판사
        publisher = item.select_one(".authPub.info_pub").get_text()
        # 출간일
        pub_date = item.select_one(".authPub.info_date").get_text()
        # 책가격
        book_price = int(item.select_one("div.info_row.info_price em.yes_b").get_text().replace(",", ""))
        # 사용자 리뷰수
        n_reviews = int(item.select_one(".txC_blue").get_text().replace(",", "")) if item.select_one(".txC_blue") != None else 0
        # 리뷰 평점
        rating = float(item.select_one(".rating_grade em.yes_b").get_text()) if item.select_one(".rating_grade em.yes_b") != None else 0
        result['book_id'] = book_id
        result['kind'] = kind
        result['book_title'] = book_title
        result['detail_link'] = detail_link
        result = result | author
        result['publisher'] = publisher
        result['pub_date'] = pub_date
        result['book_price'] = book_price
        result['n_reviews'] = n_reviews
        result['rating'] = rating
        all_result.append(result)

            
#         time.sleep(3)

dir_name = "book_data"
if not os.path.exists(dir_name):
        os.mkdir(dir_name)
        print(f"{dir_name}을 생성했습니다.")        
        
df = pd.DataFrame(all_result)
df.to_csv(f"./{dir_name}/yes24_{today}_best.csv")

NameError: name 'auth_extract' is not defined

# yes24_수집일_best.csv 파일에서 상세 링크만 가져와서 상세 페이지에 방문하고
- book_id
- 쪽수, 무게, 크기 (각각 따로 수집)
- 카테고리 분류 까지 모으기
- yes24_수집일_best_detail.csv 로 저장하기

In [43]:
import os
import time
import requests
import pandas as pd
from bs4 import BeautifulSoup as bs

from myfunc import text_clean #myfunc의 text_clean함수 가져오기

In [44]:
from datetime import datetime

In [45]:
today=datetime.today()
print(today)

2026-01-27 10:21:49.836200


In [19]:
data = pd.read_csv("./book_data/yes24_20260127_best.csv")
detail_links = data['detail_link']
detail_links

0      https://www.yes24.com/product/goods/172506508
1      https://www.yes24.com/product/goods/164510819
2      https://www.yes24.com/product/goods/168226997
3      https://www.yes24.com/product/goods/148032819
4      https://www.yes24.com/product/goods/167503385
                           ...                      
994    https://www.yes24.com/product/goods/124923518
995    https://www.yes24.com/product/goods/122865851
996    https://www.yes24.com/product/goods/122532297
997    https://www.yes24.com/product/goods/116745219
998    https://www.yes24.com/product/goods/116599481
Name: detail_link, Length: 999, dtype: object

In [20]:
all_detail_info = []
for idx, url in enumerate(detail_links):
    print(f"{idx}/{len(detail_links)}   수집중", end="\r")
    r = requests.get(url)
    soup = bs(r.content, "lxml")
    
    book_info ={}
    
    # book_id
    book_id = url.split("/")[-1]
    book_info['book_id'] = book_id 
    
    # 쪽수, 무게, 크기
    for row in soup.select("tbody.b_size tr"):
        if "쪽수" in row.select_one("th").get_text():
            pummoc_info = row.select_one("td").get_text().split("|")
    
    print(pummoc_info)
    for item in pummoc_info:
        temp = item.strip()
        if temp[-1] == "쪽":
            book_info["페이지"] = temp
        elif temp[-1] == "g":
            book_info['무게'] = temp
        elif temp[-1] == "m":
            book_info['크기'] = temp

    #카테고리 분류
    category = soup.select_one("#infoset_goodsCate .yesAlertLi > li").get_text(strip=True).split(">")    
    book_info['카테고리'] = category
   
    all_detail_info.append(book_info)
    
#     time.sleep(3)

df2 = pd.DataFrame(all_detail_info)
df2

['372쪽 ', ' 584g ', ' 140*210*27mm']
['248쪽 ', ' 336g ', ' 128*188*17mm']
['260쪽 ', ' 408g ', ' 152*225*16mm']
['296쪽 ', ' 524g ', ' 152*225*20mm']
['604쪽 ', ' 856g ', ' 152*225*29mm']
['33쪽 ', ' 270g ', ' 194*237*9mm']
['352쪽 ', ' 528g ', ' 146*210*23mm']
['392쪽 ', ' 235*310*30mm']
['272쪽 ', ' 420g ', ' 135*210*18mm']
['544쪽 ', ' 1,368g ', ' 210*280*24mm']
['202쪽 ', ' 202g ', ' 128*188*20mm']
['320쪽 ', ' 133*200*30mm']
['224쪽 ', ' 286g ', ' 128*182*15mm']
['460쪽 ', ' 1,184g ', ' 210*280*21mm']
['199쪽 ', ' 107*159*20mm']
['308쪽 ', ' 398g ', ' 128*188*30mm']
['304쪽 ', ' 300*225*30mm']
['232쪽 ', ' 272g ', ' 130*200*14mm']
['324쪽 ', ' 300*225*30mm']
['368쪽 ', ' 392g ', ' 128*188*22mm']
['260쪽 ', ' 120*189*20mm']
['200쪽 ', ' 272g ', ' 128*182*13mm']
['556쪽 ', ' 701g ', ' 148*210*24mm']
['224쪽 ', ' 352g ', ' 128*188*16mm']
['288쪽 ', ' 364g ', ' 128*188*16mm']
['404쪽 ', ' 728g ', ' 152*223*25mm']
['176쪽 ', ' 176g ', ' 128*188*20mm']
['348쪽 ', ' 152*225*30mm']
['218쪽 ', ' 605*440*30mm']
['416

KeyboardInterrupt: 

In [21]:
df2 = pd.DataFrame(all_detail_info)

dir_name = "book_data"
if not os.path.exists(dir_name):
        os.mkdir(dir_name)
        print(f"{dir_name}을 생성했습니다.")        
        

df2.to_csv(f"./{dir_name}/yes24_{today}_best_detail.csv")
df2

,book_id,페이지,무게,크기,카테고리
0,172506508,372쪽,584g,140*210*27mm,"[국내도서, 경제 경영, 투자/재테크, 재무/자산관리, 재테크일반]"
1,164510819,248쪽,336g,128*188*17mm,"[국내도서, 소설/시/희곡, 일본소설, 일본 장편소설]"
2,168226997,260쪽,408g,152*225*16mm,"[국내도서, 경제 경영, 투자/재테크, 주식/ETF, 국내주식]"
3,148032819,296쪽,524g,152*225*20mm,"[국내도서, 경제 경영, 투자/재테크, 재무/자산관리, 연금]"
4,167503385,604쪽,856g,152*225*29mm,"[국내도서, 경제 경영, 경제/경제학, 경제상식/경제이야기]"
5,27582352,33쪽,270g,194*237*9mm,"[국내도서, 유아, 4-6세, 4-6세 다른나라 그림책]"
6,166390503,352쪽,528g,146*210*23mm,"[국내도서, 인문, 인문/교양, 교양으로 읽는 인문]"
7,168393009,392쪽,NaN,235*310*30mm,"[국내도서, 수험서 자격증, 한국사능력검정시험, 심화(1,2,3급)]"
8,147276346,272쪽,420g,135*210*18mm,"[국내도서, 에세이, 한국 에세이]"
9,168776448,544쪽,"1,368g",210*280*24mm,"[국내도서, 국어 외국어 사전, 영어, 토익/TOEIC, 독해/문법/RC]"


In [22]:
df2 = pd.DataFrame(all_detail_info)

dir_name = "book_data"
if not os.path.exists(dir_name):
        os.mkdir(dir_name)
        print(f"{dir_name}을 생성했습니다.")        
        

df2.to_csv(f"./{dir_name}/yes24_{today}_best_detail.csv")
df2

,book_id,페이지,무게,크기,카테고리
0,172506508,372쪽,584g,140*210*27mm,"[국내도서, 경제 경영, 투자/재테크, 재무/자산관리, 재테크일반]"
1,164510819,248쪽,336g,128*188*17mm,"[국내도서, 소설/시/희곡, 일본소설, 일본 장편소설]"
2,168226997,260쪽,408g,152*225*16mm,"[국내도서, 경제 경영, 투자/재테크, 주식/ETF, 국내주식]"
3,148032819,296쪽,524g,152*225*20mm,"[국내도서, 경제 경영, 투자/재테크, 재무/자산관리, 연금]"
4,167503385,604쪽,856g,152*225*29mm,"[국내도서, 경제 경영, 경제/경제학, 경제상식/경제이야기]"
5,27582352,33쪽,270g,194*237*9mm,"[국내도서, 유아, 4-6세, 4-6세 다른나라 그림책]"
6,166390503,352쪽,528g,146*210*23mm,"[국내도서, 인문, 인문/교양, 교양으로 읽는 인문]"
7,168393009,392쪽,NaN,235*310*30mm,"[국내도서, 수험서 자격증, 한국사능력검정시험, 심화(1,2,3급)]"
8,147276346,272쪽,420g,135*210*18mm,"[국내도서, 에세이, 한국 에세이]"
9,168776448,544쪽,"1,368g",210*280*24mm,"[국내도서, 국어 외국어 사전, 영어, 토익/TOEIC, 독해/문법/RC]"
